In [0]:
#with tensorflow, model progress can be saved during and after training. This means a model can resume where it left off and avoid training again. Different API
#have different ways to save models. This is the way to do it for the keras API.

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.9.0-rc2'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 1s 0us/step


In [0]:
#only using a subset of the data
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [15]:
#create the model wrap this in a function for reproducibility

def create_model():

  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Dense(512, activation = 'relu', input_shape= (784,)))
  model.add(tf.keras.layers.Dropout(0.2))
  model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

  #compile the model

  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

  
  return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
#keras provides a way to automatically save checkpoints during and at the end of the training so you can use a model without having to retrain it.

#tf.keras.callbacks.ModelCheckpoint is a callback that does just this

#create a path to where you want to save the parameters
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#this instantiation of the callbacks method wil be used when fitting the model to the data
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only = True, verbose = 1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10, validation_data = (test_images, test_labels), callbacks = [cp_callback]) #cp_callback is pass into the callbacks arg


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 413us/step - loss: 2.2917 - acc: 0.2530 - val_loss: 2.0713 - val_acc: 0.4620

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 218us/step - loss: 2.2566 - acc: 0.3980 - val_loss: 4.0301 - val_acc: 0.4450

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 223us/step - loss: 2.1987 - acc: 0.4030 - val_loss: 5.8928 - val_acc: 0.4330

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 213us/step - loss: 2.1126 - acc: 0.4850 - val_loss: 7.4625 - val_acc: 0.4140

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 233us/step - loss: 2.0021 - acc: 0.4830 - val_loss: 8.6179 - val_acc: 0.3810

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=====

In [18]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 154us/step
Untrained model, accuracy: 11.70%


In [19]:
#load the weights from the checkpoint path
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 44us/step
Restored model, accuracy: 48.20%


In [20]:
#saving the entire model contains the weight values, the model config, and the optimizer config

model = create_model()

model.fit(train_images, train_labels, epochs = 5)

#save the entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 386us/step - loss: 2.2912 - acc: 0.2190
Epoch 2/5
1000/1000 [==============================] - 0s 191us/step - loss: 2.2558 - acc: 0.3120
Epoch 3/5
1000/1000 [==============================] - 0s 188us/step - loss: 2.1970 - acc: 0.4500
Epoch 4/5
1000/1000 [==============================] - 0s 183us/step - loss: 2.1102 - acc: 0.4830
Epoch 5/5
1000/1000 [==============================] - 0s 193us/step - loss: 1.9947 - acc: 0.5170


In [21]:
#recreate the same model from that file

new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 119us/step
Restored model, accuracy: 49.70%
